In [179]:
#Import the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sci

import sklearn.ensemble as se
import sklearn.model_selection as ms
import sklearn.metrics as sm
import sklearn.preprocessing as sp
import sklearn.impute as si
import seaborn as sns

import math

In [180]:
#load the raw dataset
pd.set_option('display.max_columns',None)
rawdata = pd.read_excel('Rawdata.xlsx')
rawdata.describe(include='all')

,Reference,Station,Latitude,Longitude,Depth,Temperature,Salinity,Density,O2,AT,CT,pH,pCO2,NO3,PO4,SiOH4,N2O,d15n,d18O,d15,d15Nb,SP,d15NO3,d18O-NO3,NO2,d15NO2,d18O-NO2
count,2143.000000,2143.0,2143.000000,2143.000000,1944.000000,1634.000000,1566.000000,1580.000000,1726.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,2083.000000,2026.000000,2121.000000,1928.000000,1928.000000,2139.000000,481.000000,481.000000,1119.000000,180.000000,179.000000
unique,NaN,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,6.189921,NaN,2.484248,-41.510641,487.580863,10.460081,34.577301,26.025859,105.812772,2316.207640,2222.459345,7.744936,864.187989,26.590524,2.134546,45.025273,32.789765,7.719646,54.165031,18.657786,-3.017231,21.334488,12.371139,10.994172,0.371242,-14.234812,14.526998
std,5.056160,NaN,29.932197,110.952190,645.852242,7.272478,0.658884,2.289946,101.934567,49.617404,113.508704,0.178515,349.916244,13.824633,0.946320,42.744079,27.906709,3.102976,13.510201,7.482752,4.216389,9.946216,4.961233,5.568101,1.007430,11.015585,2.734418
min,1.000000,NaN,-68.990000,-179.890909,0.000000,-1.743020,27.434599,12.046557,-0.080000,1993.526319,1840.403859,7.501403,210.335424,-0.355636,0.006641,0.839811,1.307829,-11.039096,12.400000,-1.081315,-33.884285,-9.419766,3.057646,1.601334,-0.040000,-34.259794,4.100431
25%,3.000000,NaN,-16.000000,-107.500500,65.000000,4.332500,34.520000,25.736032,5.765500,2292.009439,2149.157796,7.583326,507.243393,18.631376,1.538043,10.579253,13.335386,6.042990,45.688984,14.007500,-5.076187,14.930997,8.348434,6.822577,0.010000,-22.925794,13.496266
50%,3.000000,NaN,2.500000,-102.997000,203.950000,9.586850,34.628800,26.730000,77.650000,2304.932757,2261.862411,7.694636,877.487955,28.816097,2.438862,28.965347,24.932515,7.360000,49.850000,18.205110,-2.990000,21.260000,11.580000,10.704900,0.020000,-17.435000,14.840000
75%,10.000000,NaN,19.000000,-70.644440,700.425000,14.953500,34.778925,27.308950,198.814063,2344.954721,2296.582487,7.906677,1235.030928,38.435304,2.825251,73.323137,44.665909,8.905990,56.795092,21.916021,-0.840000,25.820000,15.676431,15.309712,0.195000,-3.092057,15.902867


In [ ]:
rawdata.columns

In [ ]:
#Missing value of each feature and target
rawdata.iloc[:,2:27].isnull().sum()

In [ ]:
#Boxplot 
a=0
fig = plt.figure(figsize=(18,20))
#data1 = pd.concat([rawdata.iloc[:,7:10],rawdata.iloc[:,11:18]],axis=1)
data1 = pd.concat([rawdata.iloc[:,2:26]],axis=1)
for i in range(24):
    axes = fig.add_subplot(4,6,a+1)
    axes = sns.boxplot(y=data1.iloc[:,i],orient="v",color="darkgreen",width=0.5)
    axes.tick_params(labelsize=12)
    axes.set_ylabel(data1.columns[i],size=12)
    plt.subplots_adjust(wspace=0.4, hspace=0.1)
    a=a+1
plt.savefig("figure_1.pdf")

In [ ]:
#Correlation heatmap
plt.figure(figsize=(10,8))
h = sns.heatmap(data1.corr(method="spearman"), vmax=1, vmin=-0.8,
            cmap='YlGn',cbar=True,
            annot=True, fmt='.2f', annot_kws={'size':8, 'fontname':'Arial'})
plt.xticks(rotation = 90, fontsize= 10, fontname = "Arial")
plt.yticks(rotation = 0, fontsize= 10, fontname = "Arial")
#plt.show()
plt.savefig("figure_2.pdf")

In [186]:
#Extract the feature for the following missing value imputation
df = rawdata.iloc[:, 4:17]
df.describe()

,Depth,Temperature,Salinity,Density,O2,AT,CT,pH,pCO2,NO3,PO4,SiOH4,N2O
count,1944.000000,1634.000000,1566.000000,1580.000000,1726.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,1365.000000,2083.000000
mean,487.580863,10.460081,34.577301,26.025859,105.812772,2316.207640,2222.459345,7.744936,864.187989,26.590524,2.134546,45.025273,32.789765
std,645.852242,7.272478,0.658884,2.289946,101.934567,49.617404,113.508704,0.178515,349.916244,13.824633,0.946320,42.744079,27.906709
min,0.000000,-1.743020,27.434599,12.046557,-0.080000,1993.526319,1840.403859,7.501403,210.335424,-0.355636,0.006641,0.839811,1.307829
25%,65.000000,4.332500,34.520000,25.736032,5.765500,2292.009439,2149.157796,7.583326,507.243393,18.631376,1.538043,10.579253,13.335386
50%,203.950000,9.586850,34.628800,26.730000,77.650000,2304.932757,2261.862411,7.694636,877.487955,28.816097,2.438862,28.965347,24.932515
75%,700.425000,14.953500,34.778925,27.308950,198.814063,2344.954721,2296.582487,7.906677,1235.030928,38.435304,2.825251,73.323137,44.665909
max,5117.200000,29.480000,36.570000,27.860000,400.865801,2456.239480,2414.440943,8.146143,1364.524934,46.753753,3.500509,197.908077,226.057678


## Missforest imputation

In [187]:
len(df.columns)

13

In [ ]:
train_idx = []
test_idx = []
df_iter = []
gamma = []
impute_df = []
for b in range(len(df.columns)):
    #Extract the index of the missing value and non-missing value
    train_idx.append(df.iloc[:,b].index[np.where(df.iloc[:,b].notnull())[0]])
    test_idx.append(df.iloc[:,b].index[np.where(df.iloc[:,b].isnull())[0]])

sortindex = np.argsort(df.isnull().sum(axis=0)).values
df_miss = df
#Initialize the stopping criterion
gp=[1,1,1,1,1,1,1,1,1,1,1,1,1]
#Set the iterations 
for a in range(5):
    print("iterations:{}".format(a))
    #In the first imputation, the dataset is imputed by the mean values
    if a < 1:
        df_z = si.SimpleImputer(missing_values = np.nan, strategy='mean').fit_transform(df_miss)
        df_z = pd.DataFrame(df_z, columns = df_miss.columns)
    #After the first imputation, the dataset is replaced by the former imputed dataset  
    else:
        df_z = df_miss.copy()
    
    for i in np.delete(sortindex,0):
        fillc = df_z.iloc[:,i]
        df_new = df_z.iloc[:,df_z.columns != df_z.columns[i]]     
        #Split the dataset
        Ytrain = fillc[train_idx[i]]
        Ytest = fillc[test_idx[i]]
        Xtrain = df_new.loc[train_idx[i],:]
        Xtest = df_new.loc[test_idx[i],:]  
        #Apply the random forest with defualt hyperparameters to impute the missing values
        rfr_impute = se.RandomForestRegressor(random_state=42,oob_score=True)
        rfr_impute = rfr_impute.fit(Xtrain, Ytrain)  
        #Apply cross-validation to explore the imputation performance 
        cv = ms.KFold(n_splits=5, shuffle=True, random_state=42)
        scores = ms.cross_val_score(rfr_impute, Xtrain, Ytrain , cv=cv, scoring='r2')
        cv_R2 = np.mean(scores)
        #Impute
        Ypredict = rfr_impute.predict(Xtest)
        impute_df.append(Ypredict)
        #The imputed feature is filled in the raw dataset
        df_z.loc[test_idx[i],df_z.columns[i]]=Ypredict 
        print(df_miss.columns[i])
        print("Train R2: %.4f" %(rfr_impute.score(Xtrain,Ytrain)))  
        print("CV R2: %.4f (+/- %.4f)" %(cv_R2, scores.std())) 
        print("Oob score: %.4f" %(rfr_impute.oob_score_)) 
        
        
    #Examine the change of the stopping criterion after imputation
    if a < 1:
        yita = sum(np.array((df_z - df_miss.fillna(value=0))**2))/ sum((np.array(df_z)**2))
    else:
        yita = sum(np.array((df_z - df_miss)**2))/ sum((np.array(df_z)**2))
    print(yita)

    if ((yita - gp) >  0).any():
        break
    gp = yita 
    gamma.append(yita)
    df_iter.append(df_z)
    df_miss = df_z
print("The iteration is converged in:{}".format(a-1))

In [ ]:
df_iter

In [ ]:
#Plot the change of data distribution after each imputation
a=0
implabel=[ 'Depth', 'Temperature', 'Salinity', 'Density', 'O2', 'AT', 'CT', 'pH', 'pCO2', 'NO3', 'PO4', 'SiOH4', 'N2O']
fig = plt.figure(figsize=(15,8))
for i in sortindex:
    axes = fig.add_subplot(3,5,a+1)
    axes = sns.kdeplot(df.iloc[:,i],color='black',linewidth=1, linestyle='--')
    axes = sns.kdeplot(df_iter[0].iloc[:,i],color='green',linewidth=1)
    axes = sns.kdeplot(df_iter[1].iloc[:,i],color='darkgreen',linewidth=2)
    axes.tick_params(labelsize=14)
    axes.set_xlabel(implabel[i],size=16) 
    axes.set_ylabel('Density',size=16)
    plt.subplots_adjust(wspace=0.32, hspace=0.3)
    a=a+1
plt.legend(['raw data','1st imputed data', '2nd imputed data'],fontsize=10)
plt.savefig("figure_3.pdf")

In [191]:
#Based on the iterative imputation, the data with second imputation is extracted 
df_iter[1].describe()

,Depth,Temperature,Salinity,Density,O2,AT,CT,pH,pCO2,NO3,PO4,SiOH4,N2O
count,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000
mean,479.530756,11.137797,34.601602,26.036709,103.096521,2311.867419,2220.966661,7.748110,846.410084,27.192629,2.097281,40.252332,32.789765
std,630.777420,7.329099,0.624860,2.040484,97.653462,41.673010,91.326575,0.154306,314.775392,11.361120,0.770417,35.449941,27.513083
min,0.000000,-1.743020,27.434599,12.046557,-0.080000,1993.526319,1840.403859,7.501403,210.335424,-0.355636,0.006641,0.839811,1.307829
25%,60.400000,4.817320,34.530000,25.662895,11.907500,2296.479282,2196.660840,7.619858,544.554811,23.423590,1.760589,18.782238,13.536400
50%,204.900000,10.563496,34.640000,26.553022,76.182137,2306.012564,2229.012397,7.708984,847.621553,28.816909,2.184625,29.695503,26.015600
75%,696.889000,15.614000,34.840250,27.219050,194.218750,2321.304786,2274.615883,7.876074,1113.372751,33.402288,2.635932,48.246675,44.125107
max,5117.200000,29.480000,36.570000,27.860000,400.865801,2456.239480,2414.440943,8.146143,1364.524934,46.753753,3.500509,197.908077,226.057678


In [ ]:
#Rearrange the dataset
df1 = pd.concat([rawdata.iloc[:,:4],df_iter[1],rawdata.iloc[:,17:26]],axis=1)
#Delete the bad data point
df2 = df1
display(df2.columns)
display(df2.describe(include='all'))

In [202]:
#Export the preprocessed data
df2.to_excel('Preprocessed_data.xlsx')

In [ ]:
#Missing value of each feature and target
df2.iloc[:,2:27].isnull().sum()